<center><h1>Numerical Analysis</h1></center>
<center><h2>2023/1</h2></center>
<center><h3>Fernando Deeke Sasse</h3></center>
<center><h3>CCT - UDESC</h3></center>
<center><h2>Gaussian Elimination - 2</h2></center>

### 1. A different view of the Gaussian elimination method

As we have seen previously the Gaussian elimination method is the most widely used direct method for solving linear systems and is the basis for all variants of elimination methods. We will now look again at the two basic steps:

1. Elimination phase: we perform the elimination of the elements below the main diagonal of the augmented matrix $[A|B]$ by means of elementary row operations, until we obtain the echelon form.
2. Backsubstitution: we simply start solving recursively the resulting equations backwards.

We describe each of these steps below, now using elimination factors, better suited for the algorithmic implementation of the method. 

### 2. Elimination phase
We call *pivot row* the row above which all elements below the main diagonal have already been zeroed.
In the expanded matrix, let the pivot row be $L_k$ and a typical line $L_i$ below to be transformed, in order to zero the element $a_{ik}$. To do so, we must multiply the pivot row by $\lambda = a_{ik}/a_{kk}$ and subtract the result from $L_i$, that is,

$$
L_i \rightarrow L_i - \lambda L_k\,.
$$

The row elements $L_i$ are updated as follows:

\begin{align}
&a_{ij} \leftarrow a_{ij} - \lambda a_{kj}\,\qquad j=k, \ldots , n\\
&b_i \leftarrow b_i - \lambda b_k\,.
\end{align}

As $a_{ik}=0$ by construction, to save computational time, it doesn't need to be calculated, so we can take $j=k+1, \ldots , n$.

The index $k$ is the index of the pivot row, so that $k=1, \ldots, n-1$ The index $i$ designates the row to be transformed, so that $i=k+1,\ \ldots, n$.

Remember that when using range(i,j), the actual range is $1, 2, \ldots, j-1$. For example,

In [ ]:
import numpy as np
c=np.array([1,2,3,4])
print(c[0])
print(c[-1])

ModuleNotFoundError: No module named 'numpy'

In [2]:
for j in range(0,3):
    print(c[j])

NameError: name 'c' is not defined

### 3. Gaussian elimination algorithm

The elimination algorithm can be performed in Python as follows:

In [331]:
def GaussElimin(a,b):
    n = len(a)
    for k in range(0,n-1): # define the index of the pivot element
        for i in range(k+1,n): # goes along the row below the pivot row and goes up to n-1 
            lam = a[i,k]/a[k,k]
            a[i,k+1:n] = a[i,k+1:n] - lam*a[k,k+1:n] # update the elements of row i
            a[i,k]=0
            b[i] = b[i] - lam*b[k]
    return  np.hstack([a,b])

Let us test the function:

In [13]:
A = np.array([[6.,1.,2.,4.],[5.,11.,-3.,2.],[-3.,4.,3.,5.],[5.,2.,8.,3.]])
print(A)

[[ 6.  1.  2.  4.]
 [ 5. 11. -3.  2.]
 [-3.  4.  3.  5.]
 [ 5.  2.  8.  3.]]


In [14]:
B = np.transpose(np.array([[2.,-4.,3.,-7.]]))
print(B)

[[ 2.]
 [-4.]
 [ 3.]
 [-7.]]


Let's form the augmented matrix $M$:

In [334]:
M = np.hstack([A,B])
print(M)

[[ 6.  1.  2.  4.  2.]
 [ 5. 11. -3.  2. -4.]
 [-3.  4.  3.  5.  3.]
 [ 5.  2.  8.  3. -7.]]


Perform the Gaussian elimination:

In [335]:
M1=GaussElimin(A,B)
print(M1)

[[  6.           1.           2.           4.           2.        ]
 [  0.          10.16666667  -4.66666667  -1.33333333  -5.66666667]
 [  0.           0.           6.06557377   7.59016393   6.50819672]
 [  0.           0.           0.          -8.77567568 -15.38648649]]


### 4. Backsubstitution phase

We begin to solve the corresponding last equation of the reduced system, that is,

$$
x_n=\frac{b_n}{a_{nn}}\,.
$$

Let us now consider the backsubstitution stage where $x_n, x_{n-1}, \ldots , x_{k+1} $ have already been computed and we must compute $x_k$ from the equation $k$:

$$
a_{kk}x_k+a_{k k+1} x_{k+1}+ \cdots + a_{k n}x_n=b_k\,.
$$

Solving for $k_k$ we get

$$
x_k=\frac{1}{a_{kk}}\left(b_k-a_{k k+1} x_{k+1}- \cdots - a_{k n}x_n\right)= \frac{1}{ a_{kk}}\left(b_k-\sum_{j=k+1}^n a_{kj}x_j\right)\,.
$$

Let's implement this function in Python.

### 5. Backsubstitution algorithm
The following function takes as input a matrix of coefficients in echelon form and the column matrix of the homogeneous part. The output is the solution of the corresponding linear system.

In [348]:
def GaussRetro(a,b):
    n = len(b)
    x=np.zeros(n)
    x[n-1]=b[n-1]/a[n-1,n-1]
    for k in range(n-1,-1,-1):
        x[k] = (b[k] - np.dot(a[k,k+1:n],x[k+1:n]))/a[k,k]
    return np.transpose([x])

To understand the use of the np.dot command above we should note that for each $k$, both $a[k,k+1:n]$ and $b[k+1:n]$ are computational vectors of length $n-k $. The np.dot command causes the respective components to be multiplied and added, just like in the usual dot product. The command range(n-1,-1,-1) causes the values of $k$ to start at $n-1$ and proceed downwards until reaching $k=0$ (which precedes -1).

 Let's use the example above. Initially we slice the enlarged matrix. The matrix of coefficients is given by

In [349]:
A1 = M1[:,0:4]
A1

array([[ 6.        ,  1.        ,  2.        ,  4.        ],
       [ 0.        , 10.16666667, -4.66666667, -1.33333333],
       [ 0.        ,  0.        ,  6.06557377,  7.59016393],
       [ 0.        ,  0.        ,  0.        , -8.77567568]])

he non-homogeneous part is given by:

In [357]:
B1= M1[:,4]
B1

array([  2.        ,  -5.66666667,   6.50819672, -15.38648649])

Let us now apply the function that performs the backsubstitution: 

In [358]:
X1 = GaussRetro(A1,B1)
X1

array([[-0.32152756],
       [-0.84200801],
       [-1.1210348 ],
       [ 1.75331075]])

We can verify that this result is correct. It is important to redefine the original system, since matrices A and B have been modified throughout the process. This procedure is fundamental in Python. 

In [365]:
A = np.array([[6.,1.,2.,4.],[5.,11.,-3.,2.],[-3.,4.,3.,5.],[5.,2.,8.,3.]])
B = np.transpose(np.array([[2.,-4.,3.,-7.]]))

In [366]:
A@X1-B

array([[-4.44089210e-16],
       [-8.88178420e-16],
       [ 8.88178420e-16],
       [-3.55271368e-15]])

We can solve linear systems with direct methods using numpy's solve command:

In [360]:
import numpy.linalg as la

In [361]:
x = la.solve(A,B)
print(x)

[[-0.32152756]
 [-0.84200801]
 [-1.1210348 ]
 [ 1.75331075]]


### 6.  Complete Gaussian elimination algorithm

Let's merge the two functions into one. The attribution a[i,k]=0 can now be removed because it is irrelevant: 

In [4]:
import numpy as np

In [5]:
def GaussSolve(a,b):
    n = len(a)
    x=np.zeros(n)
    for k in range(0,n-1):
        for i in range(k+1,n):
            lam = a[i,k]/a[k,k]
            a[i,k+1:n] = a[i,k+1:n] - lam*a[k,k+1:n]
            b[i] = b[i] - lam*b[k]
    x[n-1]=b[n-1]/a[n-1,n-1]
    for k in range(n-2,-1,-1):
        x[k] = (b[k] - np.dot(a[k,k+1:n],x[k+1:n]))/a[k,k]
    return np.transpose([x])      

Let's test the complete algorithm in the previous linear system.

In [6]:
A = np.array([[6.,1.,2.,4.],[5.,11.,-3.,2.],[-3.,4.,3.,5.],[5.,2.,8.,3.]])
B = np.transpose(np.array([[2.,-4.,3.,-7.]]))

In [7]:
X1=  GaussSolve(A,B)
X1

array([[-0.32152756],
       [-0.84200801],
       [-1.1210348 ],
       [ 1.75331075]])

which is the same result obtained previously.

### 7. Example with large systems

Let's test larger random systems

In [8]:
np.random.seed(43453)
N = 10
A3 = np.random.rand(N,N)
B3 = np.random.rand(N,1)

In [9]:
X3 = GaussSolve(A3,B3)
X3

array([[-0.78014194],
       [ 1.64411949],
       [-1.13319994],
       [-0.72833755],
       [-2.02054919],
       [ 1.06249775],
       [-0.78987105],
       [ 2.26036594],
       [ 2.07989874],
       [-0.4584725 ]])

Let's check the answer by calculating the residue:

In [10]:
np.random.seed(43453)
N = 10
A3 = np.random.rand(N,N)
B3 = np.random.rand(N,1)

In [11]:
R=A3@X3-B3
R

array([[ 0.00000000e+00],
       [-1.11022302e-16],
       [ 8.88178420e-16],
       [-2.88657986e-15],
       [-7.77156117e-16],
       [ 1.11022302e-15],
       [ 2.22044605e-16],
       [-7.54951657e-15],
       [-6.32827124e-15],
       [-3.10862447e-15]])

If the system is too large it is difficult to inspect all the components. In general, we evaluate the residue by calculating a norm of the corresponding vector. The most commonly used standard is the infinite norm, it is defined as being the highest magnitude among the components:  

$$
|\mathbf{u}|_{\infty}=\max_i\{{|u_i|},i=1,\ldots,n\}.
$$

In our case,  

In [12]:
import numpy.linalg as la

In [13]:
NR = la.norm(R, np.inf)
NR

7.549516567451064e-15

Let's test the performance of the program we write with large systems.

In [14]:
np.random.seed(43453)
N = 300
A4 = np.random.rand(N,N)
B4 = np.random.rand(N,1)

In [15]:
X4 = GaussSolve(A4,B4)

Let's calculate the residue:

In [16]:
np.random.seed(43453)
N = 300
A4 = np.random.rand(N,N)
B4 = np.random.rand(N,1)

In [17]:
NR = la.norm(A4@X4-B4,np.inf)
NR

8.250622407501851e-12

Let's estimate the CPU time required to solve this system:

In [18]:
np.random.seed(43453)
N = 300
A4 = np.random.rand(N,N)
B4 = np.random.rand(N,1)

In [19]:
%%timeit
GaussSolve(A4,B4)

221 ms ± 414 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


The CPU time required using numpy's solve function is given by:

In [20]:
np.random.seed(43453)
N = 300
A4 = np.random.rand(N,N)
B4 = np.random.rand(N,1)

In [21]:
%%timeit
la.solve(A,B)

4.16 µs ± 8.14 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


As expected, the solver is much faster than our pedagogical algorithm, by almost four orders of magnitude.

### 8. Exercises

**1.** Solve step by step a random $4 \times 4$ linear system (use the first 4 digits of your cpf for the seed), describing each step of the GaussElimin and GaussRetro algorithm. Check the answer by calculating the residue.

**2.** Solve a random system $500 \times 500$ using the GaussSolve algorithm (do not show the result). Check the correctness of the result by calculating the residue norm and compare the required CPU time. Repeat the procedure using numpy's solver.  

**3** The **condition number** of a matrix $ A $, denoted by $ \kappa(A) $, measures the sensitivity of the solution of a linear system $ A \mathbf{x} = \mathbf{b} $ to such small perturbations in the matrix $ A $ or in the vector $ \mathbf{b} $. It is defined as:
$$
\kappa(A) = \|A\| \cdot \|A^{-1}\|,
$$
where $ \|A\| $ is the norm of the matrix (usually the 2-norm). A high condition number indicates that the system is ill-conditioned, i.e., small changes in the data can cause large variations in the solution. An example of an ill-conditioned matrix is ​​the Hilbert matrix $H_n$ of order $n$, defined by
$$
H_{ij} = \frac{1}{i + j + 1}, \quad i, j = 0, 1, \dots, n.
$$

We can define Python as follows:

In [1]:
def hilb(n):
    return np.array([[1/(i+j+1) for i in range(n)] for j in range(n)])
    return np.array([[1/(i+j+1) for i in range(n)] for j in range(n)])

For example,

In [5]:
hilb(4)

array([[1.        , 0.5       , 0.33333333, 0.25      ],
       [0.5       , 0.33333333, 0.25      , 0.2       ],
       [0.33333333, 0.25      , 0.2       , 0.16666667],
       [0.25      , 0.2       , 0.16666667, 0.14285714]])

(i) Solve the linear system $HX=B$, where $H$ is the Hilbert matrix $H(30)$, $30 \times 30$ and B is a column matrix with all elements having the value 1. Use the Gaussian elimination algorithm developed earlier to solve the system. Calculate the residual in both cases.

(ii) Compare the condition number of $H$ with that of a random matrix $30 \times 30$. Use the command 

(iii) Show that a small perturbation in one component of $H(30)$ causes the solution of the system defined in (i) to change drastically.


**4.** Study system instability with Hilbert matrices $n \times n$. For example, vary the value of a coefficient of B slightly, or add a small term to the matrix H. Use the solver in Numpy.

**5.** Define a system $GX=B$, $n \times n$, with $G=[g_{ij}]$ and $g_{ij}=1/(1+i+2j)$. Check if such a system is also unstable. Vary $n$. Use the solver in Numpy.

In [2]:
import numpy as np
def m5(n):
    return np.array([[1/(i+2*j+1) for i in range(n)] for j in range(n)])

In [3]:
m5(5)

array([[1.        , 0.5       , 0.33333333, 0.25      , 0.2       ],
       [0.33333333, 0.25      , 0.2       , 0.16666667, 0.14285714],
       [0.2       , 0.16666667, 0.14285714, 0.125     , 0.11111111],
       [0.14285714, 0.125     , 0.11111111, 0.1       , 0.09090909],
       [0.11111111, 0.1       , 0.09090909, 0.08333333, 0.07692308]])